# Ex 12-1. House price prediction: Multi-input model

- 집 가격에 영향을 미치는 것은 무엇이 있을까?
    - 지역
    - 크기
    - 방의 개수
    - 집이 얼마나 멋진가! 
- 집 가격을 예측하기 위해 정형데이터와 이미지 데이터를 함께 사용해보도록 하자.
![](https://www.pyimagesearch.com/wp-content/uploads/2019/01/keras_regression_cnns_houses.jpg)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Activation, Dropout, Dense, Conv2D, MaxPooling2D, Flatten, Input, concatenate
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.preprocessing import image

import numpy as np
import argparse
import locale
import os
import pandas as pd
import glob

## Data Preprocessing
### Structured data

In [ ]:
inputPath = "data/Houses Dataset/HousesInfo.txt"
cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

In [ ]:
df.head()

In [ ]:
df["zipcode"].value_counts()

- 지역(zipcode)가 몇 군데에 몰려 있음.
- 관측치가 25개보다 적은 지역은 빼고 분석을 진행하기로 함.

In [ ]:
counts = df["zipcode"].value_counts()
zipcodes = counts[counts>25].keys()
df = df.loc[df.zipcode.isin(zipcodes)]
df.zipcode.value_counts()

- zipcode를 one-hot encoding으로 변환 

In [ ]:
df.zipcode = df.zipcode.astype('category')
df = pd.concat([df, pd.get_dummies(df.zipcode)], axis=1)
df = df.drop("zipcode", axis=1)

In [ ]:
df.head()

- price를 제외한 나머지를 X로, price를 Y로 저장

In [ ]:
X = df.drop("price", axis=1)
Y = df["price"]

### Image data

- 한 집에 대해 네 장의 사진이 아래와 같은 이름으로 저장되어 있음.

In [ ]:
image.load_img("data/Houses Dataset/1_frontal.jpg")

In [ ]:
image.load_img("data/Houses Dataset/1_bedroom.jpg")

In [ ]:
image.load_img("data/Houses Dataset/1_kitchen.jpg")

In [ ]:
image.load_img("data/Houses Dataset/1_bathroom.jpg")

- 이미지를 순차적으로 불러와서 (64, 64) 픽셀로 변환
- images_bathroom, images_bedroom, images_frontal, images_kitchen의 이름으로 각 종류의 이미지를 np.array로 모아서 저장

In [ ]:
input_path = "data/Houses Dataset"

img_names = ["bathroom", "bedroom", "frontal", "kitchen"]
for i, name in enumerate(img_names):
    exec("images_%s = []" % name)
    
for i in df.index.values:
    basePath = os.path.sep.join([input_path, "{}_*".format(i + 1)])
    housePaths = sorted(list(glob.glob(basePath)))
    inputImages = []
    for housePath in housePaths:
        img = image.load_img(housePath, target_size=(64, 64))
        x = image.img_to_array(img)
        x = x/225.
        inputImages.append(x)
    
    for j, name in enumerate(img_names):
        exec("images_%s.append(inputImages[%d])" % (name, j))
    


## Train/test split

- train, test set으로 나누고 np.array로 변환

In [ ]:
from sklearn.model_selection import train_test_split
split = train_test_split(X, Y, images_bathroom, images_bedroom, images_frontal, images_kitchen, 
                         test_size=0.25, random_state=10)

In [ ]:
var_names = ["X", "Y", "bathroom", "bedroom", "frontal", "kitchen"]
for i, name in enumerate(var_names):
    exec("train_%s = np.array(split[%d])" % (name, 2*i))
    exec("test_%s = np.array(split[%d])" % (name, 2*i+1))

In [ ]:
train_X.shape, train_Y.shape, train_bathroom.shape

## Transformation

- X에 대해 train set을 기준으로 min-max transformation 적용

In [ ]:
scaler = MinMaxScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

- Y는 최대 가격을 기준으로 0-1 사이의 값으로 변환

In [ ]:
maxprice = train_Y.max()
train_Y /= maxprice
test_Y /= maxprice

## Construct Models

- 16개의 node를 가진 두 개의 dense layer를 포함하는 모형을 만들고자 한다. (16,16)을 입력으로 받아 for loop와 keras functional API를 사용하여 자동화 하는 코드를 작성해보자.

In [ ]:
inputs = Input(shape=(5,))
nodes = (16, 16)
for (i, f) in enumerate(nodes):
    if i==0:
        x = inputs
    x = Dense(f, activation="relu")(x)
model = Model(inputs, x)
model.summary()

<font color=blue>

TO DO: 아래의 사항을 반영하여 DNN 모형을 만드는 함수를 완성하시오. 
- input layer의 shape을 `dim`으로 입력받음.
- 각 dense layer의 node의 개수를 tuple형식으로 `nodes`로 입력받음.
- 각 layer의 activation은 "relu"로 고정


In [ ]:
def create_mlp(dim, nodes=(8,4)):
    # Your answer comes here
    
    
    
    
    
    
    
    return model

In [ ]:
model = create_mlp(10, (16, 8, 4))
model.summary()

<font color=blue>

TO DO: 아래의 사항을 반영하여 CNN 모형을 만드는 함수를 완성하시오. 
- input layer의 shape을 `inputshape`으로 tuple 형태를 입력받음.
- 각 Convolution layer의 filter의 수를 `filters`로 tuple 형태를 입력받음.
- 각 `Conv2D` layer 뒤에 (2,2)를 사용한 maxpooling layer를 연결함.
- `filters`에 입력된 만큼의 Conv-Maxpooling layer를 쌓은 뒤 아래의 layer를 연결함.
    - Flatten
    - Dense(16)
    - Dropout(0.5)
    - Dense(4)

In [ ]:
def create_cnn(inputshape, filters=(16, 32)):
    # Your answer comes here
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    return model

In [ ]:
model = create_cnn((32,32,3), filters=(8, 16, 32))
model.summary()

## (1) DNN model 

<font color=blue>
    
TO DO: 위에서 작성한 `create_mlp` 함수를 사용하여 `train_X`를 input으로, `train_Y`를 output으로 하는 DNN 모형을 만드시오.  8, 4의 hidden nodes를 가지는 hidden layer를 포함하도록 만드시오. 

In [ ]:
# Your answer comes here









<font color=blue>
    
TO DO: 위에서 만든 모형을 훈련하시오. 
- `train_X, train_Y`을 train set으로 사용 
- `test_X, test_Y`를 validation set으로 사용
- epoch=100
- batch size=8
- adam optimizer 사용
- 훈련 시  mae(mean absolute error)를 함께 모니터
- 훈련 과정의 loss를 확인하기 위해 model.fit을 `history`이름으로 저장

In [ ]:
# Your answer comes here













In [ ]:
min(history.history['val_mean_absolute_error'])*maxprice, min(history.history['val_loss'])

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, 101)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, train_loss, 'b+', label='train_loss')
plt.plot(epochs, val_loss, 'bo', label='val_loss')
plt.xlabel('Epochs')
plt.legend()

plt.show()

## (2) CNN model 

<font color=blue>
    
TO DO: 
- 위에서 작성한 `create_cnn` 함수를 사용하여 bathroom, bedroom, frontal, test_kitchen의 각 사진들을 input으로 받는 4 개의 개별적인 CNN 모형을 만드시오.  
- 위에서 만들어진 4개의 모형의 output을 `concatenate` layer로 묶으시오. 
- 4개의 node를 가진 dense layer를 추가하고 output layer를 추가하여 가격을 예측하는 모형을 만드시오. 

In [ ]:
cnn_model1 = 
cnn_model2 = 
cnn_model3 = 
cnn_model4 = 

# Your answer comes here










<font color=blue>
    
TO DO: 위에서 만든 모형을 훈련하시오. 
- 4개의 집 공간에 대한 train 이미지들을 입력으로 사용
- test 이미지와 Y를 validation set으로 사용 
- epoch=100
- batch size=8
- adam optimizer 사용
- 훈련 시  mae(mean absolute error)를 함께 모니터
- 훈련 과정의 loss를 확인하기 위해 model.fit을 `history`이름으로 저장

In [ ]:
# Your answer comes here












In [ ]:
min(history.history['val_mean_absolute_error'])*maxprice, min(history.history['val_loss'])

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, 101)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, train_loss, 'b+', label='train_loss')
plt.plot(epochs, val_loss, 'bo', label='val_loss')
plt.xlabel('Epochs')
plt.legend()

plt.show()

## (3) DNN + CNN


<font color=blue>
    
TO DO: 
- (1)의 DNN 모형,  (2)의 네 개의 CNN 모형을 동일하게 만드시오 
- 위에서 만들어진 5개의 모형의 output을 `concatenate` layer로 묶으시오. 
- 4개의 node를 가진 dense layer를 추가하고 output layer를 추가하여 가격을 예측하는 모형을 만드시오. 
- (1), (2)와 같이 모형을 훈련하시오.

In [ ]:
# your answer comes here



















In [ ]:
min(history.history['val_mean_absolute_error'])*maxprice, min(history.history['val_loss'])

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, 101)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, train_loss, 'b+', label='train_loss')
plt.plot(epochs, val_loss, 'bo', label='val_loss')
plt.xlabel('Epochs')
plt.legend()

plt.show()